In [1]:
import datetime
import numpy as np
import os
import pandas as pd
import warnings

In [68]:
pd.set_option('display.max_columns', 500)

In [74]:
X = pd.read_csv('./data/multiple_sources_dataset.csv')

In [75]:
X.insp_date = pd.to_datetime(X.insp_date).dt.date
X.sort_values(by=['yelp_id', 'insp_date'], ascending=[True, True], inplace=True)
violations = ['crit_viol', 'non_crit_viol', 'crit_viol_cos', 'crit_viol_rpt', 'non_crit_viol_cos', 'non_crit_viol_rpt', 'crit_viol_tbr', 'non_crit_viol_tbr']

for x in violations:
    col_name = 'prev_' + x
    X[col_name] = X.groupby('yelp_id', sort=False)[x].shift(-1)
    X[col_name] = X[col_name].fillna(0)

In [77]:
X.loc[X.insp_date >= datetime.date(year=2016,month=3,day=1)]

,doh_id,insp_rpt,name,address,phone_num,yelp_id,lat,lon,insp_date,insp_timein,insp_type,crit_viol,non_crit_viol,crit_viol_cos,crit_viol_rpt,non_crit_viol_cos,non_crit_viol_rpt,crit_viol_tbr,non_crit_viol_tbr,yelp_cat,yelp_rating,yelp_reviews,risk,insp_name,insp_badge,license_number,crime_count,311_count,construction_count,avg_high_temp,prev_crit_viol,prev_non_crit_viol,prev_crit_viol_cos,prev_crit_viol_rpt,prev_non_crit_viol_cos,prev_non_crit_viol_rpt,prev_crit_viol_tbr,prev_non_crit_viol_tbr
1341,172655,824421,A DELI,"1300 2ND ST NE WASHINGTON, DC 20002",2028428045,a-deli-washington,38.907893,-77.003800,2016-05-06,15:03:00,Routine,2,4,0,0,0,0,2,4,"[[Italian, italian], [Delis, delis], [Sandwich...",3.5,28,3,VICTOR CURRIE,88,930514000104,9,2,2,80.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
811,5626,824847,AL CROSTINO,"1324 U ST NW Washington, DC 20009",2027970523,al-crostino-washington-2,38.916762,-77.030568,2016-05-20,17:15:00,Routine,2,2,1,0,1,0,1,1,"[[Italian, italian]]",4.0,44,3,Joyce Moore,64,2747171102262,37,1,0,84.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
965,6566,824425,AL DENTE RESTAURANT,"3201 NEW MEXICO AVE NW WASHINGTON, DC 20016",2022442223,al-dente-washington,38.933044,-77.084916,2016-05-06,11:59:00,Routine,1,1,1,0,1,0,0,0,"[[Pizza, pizza], [Italian, italian]]",3.0,149,2,FANG,611,9313012000154,2,0,0,80.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1312,172369,823287,AMSTERDAM FALAFELSHOP,"2425 18TH ST NW WASHINGTON, DC 20006",2022341969,amsterdam-falafelshop-washington-4,38.921182,-77.041889,2016-03-21,11:00:00,Routine,2,1,0,0,0,0,2,1,"[[Vegetarian, vegetarian], [Middle Eastern, mi...",4.5,1612,4,Temesgen Jemaneh.,422,931368001720,16,0,7,58.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1425,187372,823388,ANKARA RESTAURANT,"1320 19TH ST NW Washington, DC 20036",2022936301,ankara-washington,38.908083,-77.043747,2016-03-23,00:50:00,Routine,5,1,0,0,1,0,5,0,"[[Mediterranean, mediterranean], [Turkish, tur...",3.5,94,2,Carolene Bailey,107,931315000141,20,0,5,64.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,716,824849,ARGONAUT,"1433 H ST NE Washington, DC 20002",2022503660,argonaut-tavern-washington,38.900035,-76.984258,2016-05-20,17:30:00,Routine,0,3,0,0,2,0,0,1,"[[Bars, bars], [American (Traditional), tradam...",3.5,372,2,Alice Jackson,54,930065004245,18,0,2,84.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,840,823877,ARMAND'S CHICAGO PIZZERIA,"226 MASSACHUSETTS AVE NE Washington, DC 20003",2025476600,armands-chicago-pizzeria-washington,38.895099,-77.002491,2016-04-14,13:25:00,Routine,6,5,0,0,0,0,6,5,"[[Pizza, pizza], [Sandwiches, sandwiches]]",3.0,96,3,Jaime Hernandez,607,931366005472,13,0,11,71.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
371,2636,824220,AROMA,"2401 PENNSYLVANIA AVE NW Washington, DC 20037",2022966383,aroma-cafe-washington,38.903421,-77.051966,2016-05-02,14:10:00,Routine,2,0,1,0,0,0,1,0,"[[Cafes, cafes]]",4.0,44,3,Denise T. Lucas,603,931339504534,11,0,2,86.82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277,2223,823821,ASTOR PIZZA,"1829 COLUMBIA RD NW Washington, DC 20009",2027457495,astor-mediterranean-washington,38.921739,-77.043770,2016-04-13,11:33:00,Routine,5,1,0,0,0,0,5,1,"[[Greek, greek], [Mediterranean, mediterranean...",4.0,238,3,Denise T. Lucas,603,931339001873,13,0,7,72.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1367,172875,825050,ATRIUM CAFE,"1201 I ST NW Washington, DC 20005",2028425858,atrium-cafe-washington-3,38.901649,-77.028560,2016-05-27,13:15:00,Routine,7,2,0,0,0,0,7,2,"[[Sandwiches, sandwiches]]",4.0,23,3,Denise T. Lucas,603,930515000001,0,0,0,73.11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
X.insp_date = pd.to_datetime(X.insp_date).dt.date
X['orig_insp_date'] = X['insp_date']

X.sort_values(by=['yelp_id', 'insp_date'], ascending=[True, True], inplace=True)

X.insp_date = [datetime.date(year=2016,month=3,day=1) if x >= datetime.date(year=2016,month=3,day=1) else x for x in X.insp_date]

X['time_diff'] = X.groupby('yelp_id', sort=False)['insp_date'].diff()

X.loc[X.time_diff.isnull(), 'time_diff'] = 0
X.time_diff = [x.days for x in X.time_diff]

X.head()

,doh_id,insp_rpt,name,address,phone_num,yelp_id,lat,lon,insp_date,insp_timein,insp_type,crit_viol,non_crit_viol,crit_viol_cos,crit_viol_rpt,non_crit_viol_cos,non_crit_viol_rpt,crit_viol_tbr,non_crit_viol_tbr,yelp_cat,yelp_rating,yelp_reviews,risk,insp_name,insp_badge,license_number,crime_count,311_count,construction_count,avg_high_temp,prev_crit_viol,prev_non_crit_viol,prev_crit_viol_cos,prev_crit_viol_rpt,prev_non_crit_viol_cos,prev_non_crit_viol_rpt,prev_crit_viol_tbr,prev_non_crit_viol_tbr,orig_insp_date,time_diff
1061,7190,404922,A & D BAR/RESTAURANT-n,"1314 9TH ST NW WASHINGTON, DC 20001",2022901804,a-and-d-bar-washington,38.907780,-77.024258,2014-10-16,14:00:00,Routine,1,1,1,0,0,0,0,1,"[[Dive Bars, divebars], [Cocktail Bars, cockta...",4.0,83,2,A. Jackson,54,931313000036,15,0,9,63.26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014-10-16,0
1026,6994,203677,A BLOOMIN SANDWICH CAFE,"2141 K ST NW Washington, DC 20037",2026590033,a-bloomin-sandwich-cafe-washington,38.902913,-77.048066,2014-05-19,11:45:00,Routine,10,4,3,0,1,0,7,3,"[[Sandwiches, sandwiches]]",4.5,3,2,Douglas Dalier,82,931912000334,15,0,6,81.03,7.0,2.0,1.0,0.0,0.0,0.0,6.0,2.0,2014-05-19,0
1027,6994,405898,A BLOOMIN SANDWICH CAFE,"2141 K ST NW Washington, DC 20037",2026590033,a-bloomin-sandwich-cafe-washington,38.902913,-77.048066,2014-11-19,11:50:00,Routine,7,2,1,0,0,0,6,2,"[[Sandwiches, sandwiches]]",4.5,3,2,ALEMAYEHU TEKLESELASSIE,29,931912000334,28,0,11,49.60,8.0,1.0,0.0,0.0,0.0,0.0,8.0,1.0,2014-11-19,184
1028,6994,814313,A BLOOMIN SANDWICH CAFE,"2141 K ST NW Washington, DC 20037",2026590033,a-bloomin-sandwich-cafe-washington,38.902913,-77.048066,2015-10-21,12:20:00,Routine,8,1,0,0,0,0,8,1,"[[Sandwiches, sandwiches]]",4.5,3,2,Temesgen Jemaneh,422,931912000334,28,0,11,68.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-10-21,336
941,6476,202812,A DELI,"1300 2ND ST NE WASHINGTON, DC 20002",2028428045,a-deli-washington,38.907893,-77.003800,2014-04-03,14:48:00,Routine,11,7,2,0,0,0,9,7,"[[Italian, italian], [Delis, delis], [Sandwich...",3.5,28,2,Laura Wildey,89,931312000118,4,0,9,70.00,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2014-04-03,0


In [73]:
X.loc[X.orig_insp_date >= datetime.date(year=2016,month=3,day=1)]

,doh_id,insp_rpt,name,address,phone_num,yelp_id,lat,lon,insp_date,insp_timein,insp_type,crit_viol,non_crit_viol,crit_viol_cos,crit_viol_rpt,non_crit_viol_cos,non_crit_viol_rpt,crit_viol_tbr,non_crit_viol_tbr,yelp_cat,yelp_rating,yelp_reviews,risk,insp_name,insp_badge,license_number,crime_count,311_count,construction_count,avg_high_temp,prev_crit_viol,prev_non_crit_viol,prev_crit_viol_cos,prev_crit_viol_rpt,prev_non_crit_viol_cos,prev_non_crit_viol_rpt,prev_crit_viol_tbr,prev_non_crit_viol_tbr,orig_insp_date,time_diff
1341,172655,824421,A DELI,"1300 2ND ST NE WASHINGTON, DC 20002",2028428045,a-deli-washington,38.907893,-77.003800,2016-03-01,15:03:00,Routine,2,4,0,0,0,0,2,4,"[[Italian, italian], [Delis, delis], [Sandwich...",3.5,28,3,VICTOR CURRIE,88,930514000104,9,2,2,80.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-05-06,278
811,5626,824847,AL CROSTINO,"1324 U ST NW Washington, DC 20009",2027970523,al-crostino-washington-2,38.916762,-77.030568,2016-03-01,17:15:00,Routine,2,2,1,0,1,0,1,1,"[[Italian, italian]]",4.0,44,3,Joyce Moore,64,2747171102262,37,1,0,84.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-05-20,0
965,6566,824425,AL DENTE RESTAURANT,"3201 NEW MEXICO AVE NW WASHINGTON, DC 20016",2022442223,al-dente-washington,38.933044,-77.084916,2016-03-01,11:59:00,Routine,1,1,1,0,1,0,0,0,"[[Pizza, pizza], [Italian, italian]]",3.0,149,2,FANG,611,9313012000154,2,0,0,80.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-05-06,467
1312,172369,823287,AMSTERDAM FALAFELSHOP,"2425 18TH ST NW WASHINGTON, DC 20006",2022341969,amsterdam-falafelshop-washington-4,38.921182,-77.041889,2016-03-01,11:00:00,Routine,2,1,0,0,0,0,2,1,"[[Vegetarian, vegetarian], [Middle Eastern, mi...",4.5,1612,4,Temesgen Jemaneh.,422,931368001720,16,0,7,58.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-03-21,162
1425,187372,823388,ANKARA RESTAURANT,"1320 19TH ST NW Washington, DC 20036",2022936301,ankara-washington,38.908083,-77.043747,2016-03-01,00:50:00,Routine,5,1,0,0,1,0,5,0,"[[Mediterranean, mediterranean], [Turkish, tur...",3.5,94,2,Carolene Bailey,107,931315000141,20,0,5,64.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-03-23,0
8,716,824849,ARGONAUT,"1433 H ST NE Washington, DC 20002",2022503660,argonaut-tavern-washington,38.900035,-76.984258,2016-03-01,17:30:00,Routine,0,3,0,0,2,0,0,1,"[[Bars, bars], [American (Traditional), tradam...",3.5,372,2,Alice Jackson,54,930065004245,18,0,2,84.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-05-20,536
25,840,823877,ARMAND'S CHICAGO PIZZERIA,"226 MASSACHUSETTS AVE NE Washington, DC 20003",2025476600,armands-chicago-pizzeria-washington,38.895099,-77.002491,2016-03-01,13:25:00,Routine,6,5,0,0,0,0,6,5,"[[Pizza, pizza], [Sandwiches, sandwiches]]",3.0,96,3,Jaime Hernandez,607,931366005472,13,0,11,71.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-04-14,341
371,2636,824220,AROMA,"2401 PENNSYLVANIA AVE NW Washington, DC 20037",2022966383,aroma-cafe-washington,38.903421,-77.051966,2016-03-01,14:10:00,Routine,2,0,1,0,0,0,1,0,"[[Cafes, cafes]]",4.0,44,3,Denise T. Lucas,603,931339504534,11,0,2,86.82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-05-02,139
277,2223,823821,ASTOR PIZZA,"1829 COLUMBIA RD NW Washington, DC 20009",2027457495,astor-mediterranean-washington,38.921739,-77.043770,2016-03-01,11:33:00,Routine,5,1,0,0,0,0,5,1,"[[Greek, greek], [Mediterranean, mediterranean...",4.0,238,3,Denise T. Lucas,603,931339001873,13,0,7,72.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-04-13,279
1367,172875,825050,ATRIUM CAFE,"1201 I ST NW Washington, DC 20005",2028425858,atrium-cafe-washington-3,38.901649,-77.028560,2016-03-01,13:15:00,Routine,7,2,0,0,0,0,7,2,"[[Sandwiches, sandwiches]]",4.0,23,3,Denise T. Lucas,603,930515000001,0,0,0,73.11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-05-27,421


In [64]:
X.insp_date = [datetime.date(year=2016,month=3,day=1) if x >= datetime.date(year=2016,month=3,day=1) else x for x in X.insp_date]

In [65]:
X.loc[X.insp_date >= datetime.date(year=2016,month=3,day=1)]

,doh_id,insp_rpt,name,address,phone_num,yelp_id,lat,lon,insp_date,insp_timein,insp_type,crit_viol,non_crit_viol,crit_viol_cos,crit_viol_rpt,non_crit_viol_cos,non_crit_viol_rpt,crit_viol_tbr,non_crit_viol_tbr,yelp_cat,yelp_rating,yelp_reviews,risk,insp_name,insp_badge,license_number,crime_count,311_count,construction_count,avg_high_temp,orig_insp_date
1341,172655,824421,A DELI,"1300 2ND ST NE WASHINGTON, DC 20002",2028428045,a-deli-washington,38.907893,-77.003800,2016-03-01,15:03:00,Routine,2,4,0,0,0,0,2,4,"[[Italian, italian], [Delis, delis], [Sandwich...",3.5,28,3,VICTOR CURRIE,88,930514000104,9,2,2,80.41,2016-05-06
811,5626,824847,AL CROSTINO,"1324 U ST NW Washington, DC 20009",2027970523,al-crostino-washington-2,38.916762,-77.030568,2016-03-01,17:15:00,Routine,2,2,1,0,1,0,1,1,"[[Italian, italian]]",4.0,44,3,Joyce Moore,64,2747171102262,37,1,0,84.14,2016-05-20
965,6566,824425,AL DENTE RESTAURANT,"3201 NEW MEXICO AVE NW WASHINGTON, DC 20016",2022442223,al-dente-washington,38.933044,-77.084916,2016-03-01,11:59:00,Routine,1,1,1,0,1,0,0,0,"[[Pizza, pizza], [Italian, italian]]",3.0,149,2,FANG,611,9313012000154,2,0,0,80.41,2016-05-06
1312,172369,823287,AMSTERDAM FALAFELSHOP,"2425 18TH ST NW WASHINGTON, DC 20006",2022341969,amsterdam-falafelshop-washington-4,38.921182,-77.041889,2016-03-01,11:00:00,Routine,2,1,0,0,0,0,2,1,"[[Vegetarian, vegetarian], [Middle Eastern, mi...",4.5,1612,4,Temesgen Jemaneh.,422,931368001720,16,0,7,58.31,2016-03-21
1425,187372,823388,ANKARA RESTAURANT,"1320 19TH ST NW Washington, DC 20036",2022936301,ankara-washington,38.908083,-77.043747,2016-03-01,00:50:00,Routine,5,1,0,0,1,0,5,0,"[[Mediterranean, mediterranean], [Turkish, tur...",3.5,94,2,Carolene Bailey,107,931315000141,20,0,5,64.85,2016-03-23
8,716,824849,ARGONAUT,"1433 H ST NE Washington, DC 20002",2022503660,argonaut-tavern-washington,38.900035,-76.984258,2016-03-01,17:30:00,Routine,0,3,0,0,2,0,0,1,"[[Bars, bars], [American (Traditional), tradam...",3.5,372,2,Alice Jackson,54,930065004245,18,0,2,84.14,2016-05-20
25,840,823877,ARMAND'S CHICAGO PIZZERIA,"226 MASSACHUSETTS AVE NE Washington, DC 20003",2025476600,armands-chicago-pizzeria-washington,38.895099,-77.002491,2016-03-01,13:25:00,Routine,6,5,0,0,0,0,6,5,"[[Pizza, pizza], [Sandwiches, sandwiches]]",3.0,96,3,Jaime Hernandez,607,931366005472,13,0,11,71.36,2016-04-14
371,2636,824220,AROMA,"2401 PENNSYLVANIA AVE NW Washington, DC 20037",2022966383,aroma-cafe-washington,38.903421,-77.051966,2016-03-01,14:10:00,Routine,2,0,1,0,0,0,1,0,"[[Cafes, cafes]]",4.0,44,3,Denise T. Lucas,603,931339504534,11,0,2,86.82,2016-05-02
277,2223,823821,ASTOR PIZZA,"1829 COLUMBIA RD NW Washington, DC 20009",2027457495,astor-mediterranean-washington,38.921739,-77.043770,2016-03-01,11:33:00,Routine,5,1,0,0,0,0,5,1,"[[Greek, greek], [Mediterranean, mediterranean...",4.0,238,3,Denise T. Lucas,603,931339001873,13,0,7,72.27,2016-04-13
1367,172875,825050,ATRIUM CAFE,"1201 I ST NW Washington, DC 20005",2028425858,atrium-cafe-washington-3,38.901649,-77.028560,2016-03-01,13:15:00,Routine,7,2,0,0,0,0,7,2,"[[Sandwiches, sandwiches]]",4.0,23,3,Denise T. Lucas,603,930515000001,0,0,0,73.11,2016-05-27


In [66]:
X['time_diff'] = X.groupby('yelp_id', sort=False)['insp_date'].diff()

In [67]:
X.head(8)

,doh_id,insp_rpt,name,address,phone_num,yelp_id,lat,lon,insp_date,insp_timein,insp_type,crit_viol,non_crit_viol,crit_viol_cos,crit_viol_rpt,non_crit_viol_cos,non_crit_viol_rpt,crit_viol_tbr,non_crit_viol_tbr,yelp_cat,yelp_rating,yelp_reviews,risk,insp_name,insp_badge,license_number,crime_count,311_count,construction_count,avg_high_temp,orig_insp_date,time_diff
1061,7190,404922,A & D BAR/RESTAURANT-n,"1314 9TH ST NW WASHINGTON, DC 20001",2022901804,a-and-d-bar-washington,38.907780,-77.024258,2014-10-16,14:00:00,Routine,1,1,1,0,0,0,0,1,"[[Dive Bars, divebars], [Cocktail Bars, cockta...",4.0,83,2,A. Jackson,54,931313000036,15,0,9,63.26,2014-10-16,NaT
1026,6994,203677,A BLOOMIN SANDWICH CAFE,"2141 K ST NW Washington, DC 20037",2026590033,a-bloomin-sandwich-cafe-washington,38.902913,-77.048066,2014-05-19,11:45:00,Routine,10,4,3,0,1,0,7,3,"[[Sandwiches, sandwiches]]",4.5,3,2,Douglas Dalier,82,931912000334,15,0,6,81.03,2014-05-19,NaT
1027,6994,405898,A BLOOMIN SANDWICH CAFE,"2141 K ST NW Washington, DC 20037",2026590033,a-bloomin-sandwich-cafe-washington,38.902913,-77.048066,2014-11-19,11:50:00,Routine,7,2,1,0,0,0,6,2,"[[Sandwiches, sandwiches]]",4.5,3,2,ALEMAYEHU TEKLESELASSIE,29,931912000334,28,0,11,49.60,2014-11-19,184 days
1028,6994,814313,A BLOOMIN SANDWICH CAFE,"2141 K ST NW Washington, DC 20037",2026590033,a-bloomin-sandwich-cafe-washington,38.902913,-77.048066,2015-10-21,12:20:00,Routine,8,1,0,0,0,0,8,1,"[[Sandwiches, sandwiches]]",4.5,3,2,Temesgen Jemaneh,422,931912000334,28,0,11,68.24,2015-10-21,336 days
941,6476,202812,A DELI,"1300 2ND ST NE WASHINGTON, DC 20002",2028428045,a-deli-washington,38.907893,-77.003800,2014-04-03,14:48:00,Routine,11,7,2,0,0,0,9,7,"[[Italian, italian], [Delis, delis], [Sandwich...",3.5,28,2,Laura Wildey,89,931312000118,4,0,9,70.00,2014-04-03,NaT
1339,172655,405585,A DELI,"1300 2ND ST NE WASHINGTON, DC 20002",2028428045,a-deli-washington,38.907893,-77.003800,2014-11-05,13:00:00,Routine,4,0,4,0,0,0,0,0,"[[Italian, italian], [Delis, delis], [Sandwich...",3.5,28,3,SHARON CAVE,124,930514000104,14,0,9,53.98,2014-11-05,216 days
1340,172655,810756,A DELI,"1300 2ND ST NE WASHINGTON, DC 20002",2028428045,a-deli-washington,38.907893,-77.003800,2015-05-28,10:15:00,Routine,12,4,2,0,0,0,10,4,"[[Italian, italian], [Delis, delis], [Sandwich...",3.5,28,3,Victoria Fletcher,87,930514000104,15,0,13,76.74,2015-05-28,204 days
1341,172655,824421,A DELI,"1300 2ND ST NE WASHINGTON, DC 20002",2028428045,a-deli-washington,38.907893,-77.003800,2016-03-01,15:03:00,Routine,2,4,0,0,0,0,2,4,"[[Italian, italian], [Delis, delis], [Sandwich...",3.5,28,3,VICTOR CURRIE,88,930514000104,9,2,2,80.41,2016-05-06,278 days


In [33]:
X['time_diff'] = X.groupby('yelp_id', sort=False)['insp_date'].diff()

In [44]:
X['predict_date'] = datetime.date(year=2016,month=3,day=1)

In [ ]:
if X.loc[X.insp_date >= datetime.date(year=2016,month=3,day=1)].time_diff 